<a href="https://colab.research.google.com/github/krish0195/ml_end_to-_end/blob/master/xg_boost_running.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Use to import pandas
import pandas as pd
#Use to import the file into google Colab drive
from google.colab import files
#Use to import io, which opens the file from the Colab drive
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics
from sklearn.metrics import roc_auc_score,roc_curve,classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import StandardScaler
std_scale=StandardScaler()
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
# This will open a widget when run that will enable you to browse the files on your local storage drive.
# the excel file after reshaping each to 1000 records:
uploaded = files.upload()

Saving Final_df.csv to Final_df.csv


In [4]:

df_fulldata=pd.read_csv('Final_df.csv')


In [5]:
#label encoding of tgt varaible
tgt_map = {    'Flexion_football':0,
               'Flexion_relaxed':1,
               'Flexion_stiffed':2,
               'Lateral_football':3,
               'Lateral_relaxed':4,
               'Lateral_stiffed':5
              }
df_fulldata["Direction"]=df_fulldata.Direction.map(tgt_map)
df_fulldata=df_fulldata.drop(columns=["time","Subject","Sr No"],axis=1)


In [6]:
x=df_fulldata.drop(columns=['Direction'])
y=df_fulldata['Direction']
df_fulldata['Direction'].value_counts()

1    10000
2    10000
3    10000
4    10000
5    10000
0    10000
Name: Direction, dtype: int64

In [7]:
#method 2:Feature selection:PCA
#standard before doing PCA for reducing dimensions:
from sklearn.preprocessing import StandardScaler
scalar=StandardScaler()
scalar.fit(x)
scaled_data = scalar.transform(x)


from sklearn.decomposition import PCA
pca = PCA(n_components=30)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
scaled_data.shape
x_pca.shape

(60000, 30)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.20, stratify = y, random_state = 43,shuffle=True)

print("x_train shape is",x_train.shape)
print("y_train shape is",y_train.shape)

print("x_test shape is",x_test.shape)
print("y_test shape is",y_test.shape)

x_train shape is (48000, 30)
y_train shape is (48000,)
x_test shape is (12000, 30)
y_test shape is (12000,)


In [9]:
#Classification Metrics:

def prediction(x_train,y_train,x_test,y_test,y_train_predict,y_test_predict):

  #Accuracy Check of Train
  train_accuracy = metrics.accuracy_score(y_train,y_train_predict)
  print("The Train Accuracy:",round(train_accuracy,3))

# Accuracy Check of Test
  test_accuracy = metrics.accuracy_score(y_test,y_test_predict)
  print("The Test Accuracy",round(test_accuracy,4))


  return train_accuracy,test_accuracy

In [10]:
#4) XG Boost calssifier
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier()

xgb_classifier.fit(x_train, y_train)
y_train_predict = xgb_classifier.predict(x_train)
y_test_predict = xgb_classifier.predict(x_test)

print("the Train and Test accuracy of XG Boost calssifier ")
XGB_train_accuracy, XGB_test_accuracy=prediction(x_train,y_train,x_test,y_test,y_train_predict,y_test_predict)

the Train and Test accuracy of XG Boost calssifier 
The Train Accuracy: 0.996
The Test Accuracy 0.9839


In [11]:
import lime
from lime import lime_tabular


ModuleNotFoundError: ignored

In [ ]:
#Comparison dataframe
index=["Accuracy"]
data = pd.DataFrame({

                     'xgb Train':[XGB_train_accuracy],
                     'xgb Test':[XGB_test_accuracy],

                     },index=index)

print(round(data,4))

In [ ]:

explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(x_train),
    feature_names=x_train.columns,
    class_names=['0', '1',"2","3","4","5"],
    mode='classification'
)

In [ ]:
exp=explainer.explain_instance(
    date_row=x_test.iloc(4),
    predict_fn=xgb_classifier.predict_proba
)
exp.show_in_notebook(show_table=True)

In [ ]:
 #4)CROSS VALIDATION-xgb Classifier

import xgboost as xgb
xgb_classifier = xgb.XGBClassifier()
cv_score_xg = cross_val_score(xgb_classifier, x, y, cv=5)
print(cv_score_xg)
mean_accuracy_xg = sum(cv_score_lr)/len(cv_score_xg)
mean_accuracy_xg = mean_accuracy_dt*100
mean_accuracy_xg = round(mean_accuracy_xg, 2)
print(mean_accuracy_xg)

In [ ]:
#mean_accuracy_list
results = pd.DataFrame({
    'Model': ['xgb'],
    'Score': [mean_accuracy_xg]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)